# requests + 正则表达式 爬取猫眼电影TOP100

## Pycharm
https://maoyan.com/board/4

In [21]:
# 基础版本 抓取数据并解析成字符串打印
import re
import requests
import lxml
from bs4 import BeautifulSoup
from requests.exceptions import RequestException


def get_one_page(url, headers):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print(response.status_code, response.headers)
            return None
    except RequestException:
        return None


def parse_one_page(html):
    # print(html)
    soup = BeautifulSoup(html, 'lxml')
    html_ = soup.find('div', class_='main')
    # print(str(html_))
    pattern = re.compile('<dd.*?board-index.*?(\d*)</i>.*?<a.*?href="(.*?)".*?>.*?board-img.*?src="(.*?)".*?'
                         'name.*?title="(.*?)".*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?'
                         'integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, str(html_))
    print(len(items))
    for r in items:
        index, url, cover, title, star, releasetime, integer, fraction = r  # releasetime 上映时间
        url = f"https://www.maoyan.com{url}"
        score = f"{integer}{fraction}"
        star = re.sub('\s', '', star)
        releasetime = re.sub('\s', '', releasetime)
        print(index, url, cover, title, star, releasetime, score)

        

def main():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "close"
    }
    url = 'https://maoyan.com/board/4'
    html = get_one_page(url, headers)
    parse_one_page(html)


if __name__ == '__main__':
    main()


10
1 https://www.maoyan.com/films/1200486 https://p0.pipi.cn/mmdb/d2dad59253751bd236338fa5bd5a27c710413.jpg?imageView2/1/w/160/h/220 我不是药神 主演：徐峥,周一围,王传君 上映时间：2018-07-05 9.6
2 https://www.maoyan.com/films/1297 https://p0.pipi.cn/mmdb/d2dad592c7e7e1d236f2aa1811a8a64794b33.jpg?imageView2/1/w/160/h/220 肖申克的救赎 主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿 上映时间：1994-09-10(加拿大) 9.5
3 https://www.maoyan.com/films/1206605 https://p0.pipi.cn/mmdb/d2dad59253751b230f21f0818a5bfd4d8679c.jpg?imageView2/1/w/160/h/220 绿皮书 主演：维果·莫腾森,马赫沙拉·阿里,琳达·卡德里尼 上映时间：2019-03-01 9.5
4 https://www.maoyan.com/films/1292 https://p0.pipi.cn/mmdb/d2dad592c7e7e1d2365bf1b63cd25951b722b.jpg?imageView2/1/w/160/h/220 海上钢琴师 主演：蒂姆·罗斯,比尔·努恩,克兰伦斯·威廉姆斯三世 上映时间：2019-11-15 9.3
5 https://www.maoyan.com/films/1211270 https://p0.pipi.cn/mmdb/d2dad592537923f0ee07acada3ac59b9f3ffb.jpg?imageView2/1/w/160/h/220 哪吒之魔童降世 主演：吕艳婷,囧森瑟夫,瀚墨 上映时间：2019-07-26 9.6
6 https://www.maoyan.com/films/1203 https://p0.pipi.cn/mmdb/d2dad592c7e7e1d236d7c3c9b6a0b65c4d19d.jpg?imageView2

---

In [ ]:
import json
import re
import requests
from time import time
from multiprocessing import Pool
from bs4 import BeautifulSoup
from requests.exceptions import RequestException


def get_one_page(url, headers):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:
            print(response.status_code, response.headers)
            return None
    except RequestException:
        return None


def parse_one_page(html):
    # print(html)
    soup = BeautifulSoup(html, 'lxml')
    html_ = soup.find('div', class_='main')
    # print(str(html_))
    pattern = re.compile('<dd.*?board-index.*?(\d*)</i>.*?<a.*?href="(.*?)".*?>.*?board-img.*?src="(.*?)".*?'
                         'name.*?title="(.*?)".*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?'
                         'integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, str(html_))
    print(len(items))
    for r in items:
        # index, url, cover, title, actor, releasetime, integer, fraction = r  # releasetime 上映时间
        # url = f"https://www.maoyan.com{url}"
        # score = f"{integer}{fraction}"
        # actor = re.sub('\s', '', actor)
        # releasetime = re.sub('\s', '', releasetime)
        # print(index, url, cover, title, star, releasetime, score)

        # 或者
        yield {
            'index': r[0],
            'title': r[3],
            'actor': r[4].strip()[3:],  # "主演：徐峥" 👉 "徐峥"
            'releasetime': r[5].strip()[5:],  # 同理: 上映时间：2020-01-03 9.3 👉 2020-01-03 9.3
            'score': r[6] + r[7],
            'url': f"https://www.maoyan.com{r[1]}",
            'cover': r[2]
        }


# def write_to_file(content):
#     # 'a' 表示追加
#     with open('result_maoyan.txt', 'a', encoding='utf8') as f:
#         f.write(json.dumps(content, ensure_ascii=False) + '\n')
#         f.close()


def main(offset):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "close"
    }
    url = f'https://maoyan.com/board/4?offset={offset}'
    html = get_one_page(url, headers)
    for item in parse_one_page(html):
        print(item)
#         write_to_file(item)


if __name__ == '__main__':
    # 猫眼 TOP100 共 10页数据每页十条
    start = time()

    # 耗时：2.844808340072632秒
    # for i in range(10):
    #     main(i * 10)

    # 耗时：1.3851382732391357秒
    pool = Pool()
    pool.map(main, [i * 10 for i in range(10)])

    end = time()
    print("耗时：" + str(end - start) + "秒")
